In [4]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
data = kagglehub.dataset_download('xhlulu/140k-real-and-fake-faces')
model = kagglehub.dataset_download('raphaeldamasceno/modelos')

print('\nData source import complete.')

Resuming download from 1071644672 bytes (2952911046 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/xhlulu/140k-real-and-fake-faces?dataset_version_number=2 (1071644672/4024555718) bytes left.


100%|█████████████████████████████████████████████████████████████████████████████| 3.75G/3.75G [03:24<00:00, 14.4MB/s]

Extracting files...


100%|███████████████████████████████████████████████████████████████████████████████| 507M/507M [00:34<00:00, 15.3MB/s]

Extracting files...



Data source import complete.


In [5]:
data

'C:\\Users\\Mishka\\.cache\\kagglehub\\datasets\\xhlulu\\140k-real-and-fake-faces\\versions\\2'

In [41]:
import kagglehub
modelDownloaded = kagglehub.dataset_download('raphaeldamasceno/modelos')

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [18]:
!pip install timm

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.4 MB 1.2 MB/s eta 0:00:02
   ----------------- ---------------------- 1.0/2.4 MB 1.9 MB/s eta 0:00:01
   ------------------------------- -------- 1.8/2.4 MB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 2.5 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
!pip install pandas matplotlib pillow seaborn torch torchvision tqdm scikit-learn

In [4]:
os.listdir()

['.ipynb_checkpoints',
 'app.py',
 'project',
 'SEAI_Project.ipynb',
 '__pycache__']

In [6]:
os.

'C:\\Users\\Mishka\\OneDrive\\Desktop\\seai'

In [42]:
import pandas as pd
import random
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sb
import torch
from torchvision import models
from torch import nn
from torch import optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models.vision_transformer import vit_b_16, ViT_B_16_Weights
from torchvision.utils import make_grid
from torch.autograd import Variable
import torch.optim.lr_scheduler as lr_scheduler
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score, precision_recall_fscore_support
import shutil  ##offers high-level operation on a file like a copy, create, and remote operation on the file
import os
import warnings
warnings.filterwarnings("ignore")

code_dir = "/project/working/code"
model_dir = "/project/working/model"
output_dir = "/project/working/output"

if not os.path.exists(code_dir):
    os.makedirs(code_dir)

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

shutil.copyfile(src=os.path.join(modelDownloaded, "convnext.py"),
                dst="/project/working/code/convnext.py")
shutil.copyfile(src=os.path.join(modelDownloaded,"convnext_tiny_1k_224_ema.pth"),
                dst="/project/working/model/convnext_tiny_1k_224_ema.pth")

os.chdir("/project/working/code")


from convnext import ConvNeXt


# **2. Database**

## **2.1. Network Architectures**

In [20]:
os.getcwd()

'C:\\project\\working\\code'

In [43]:
#Instantiating the ConvNeXt model
from convnext import ConvNeXt
def ConvNeXt_model():
    model_conv=ConvNeXt()
    state_dict = torch.load('/project/working/model/convnext_tiny_1k_224_ema.pth')
    model_conv.load_state_dict(state_dict["model"])

    return model_conv

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [9]:
criterion = nn.CrossEntropyLoss()

In [10]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [11]:
train_acc, test_acc, val_loss, val_loss = [], [], [], []
train_precision, val_precision, train_recall, val_recall = [], [], [], []
train_f1, val_f1 = [], []
df = pd.DataFrame(columns=['Epoch', 'Train ACC', 'Train Loss', 'Train F1', 'Val ACC', 'Val Loss', 'Val F1'])

# **3. Methodology**

## **3.1.  Data Augmentation**

### **3.1.1. Image Processing**

In [32]:
def full_data_transform(data_fraction, batch_size):
    # Data transformations
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(256),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])


    local_arquivos="C:/Users/Mishka/.cache/kagglehub/datasets/xhlulu/140k-real-and-fake-faces/versions/2/real_vs_fake/real-vs-fake"
    full_train_dataset = ImageFolder(local_arquivos + "/train", transform=transform)
    full_val_dataset = ImageFolder(local_arquivos + "/valid", transform=transform)

    # Determine the number of objects to be selected
    num_train_data = int(len(full_train_dataset) * data_fraction)
    num_val_data = int(len(full_val_dataset) * data_fraction)



    # Randomly select objects for the datasets
    train_indices = random.sample(range(len(full_train_dataset)), num_train_data)
    val_indices = random.sample(range(len(full_val_dataset)), num_val_data)

    # Create datasets with randomly selected objects
    train_dataset = torch.utils.data.Subset(full_train_dataset, train_indices)
    val_dataset = torch.utils.data.Subset(full_val_dataset, val_indices)

    # Create dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=int(batch_size/2), shuffle=False)


    return train_dataloader, val_dataloader


## **3.2. Training and Testing**

In [33]:
 # Generic training function
def train(model, dataloader, criterion, optimizer, scheduler, device, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    y_true,y_pred=[], []

    loop = tqdm(enumerate(dataloader), total=len(dataloader))
    for batch_idx, (images, labels) in loop:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad() #Clears the gradients of all optimized tensors before performing backpropagation.
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        predicted = outputs.argmax(dim = 1)

        y_true.extend(labels.cpu().tolist())
        y_pred.extend(predicted.cpu().tolist())

        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        loop.set_description(f"[Epoch {(epoch+1)}]")
        loop.set_postfix(loss=loss.item())

    if scheduler:
        scheduler.step()

    train_loss = running_loss / len(dataloader.dataset)
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro',zero_division=0)

    print(f"Train Loss: {train_loss:.6f} | Train Accuracy: {(accuracy * 100):.2f}% | Train F1-Score: {f1:.6f}")

    model_name= f'convnext.pth'
    torch.save(model.state_dict(), os.path.join('/project/working/model', model_name))

    return train_loss, accuracy, f1

In [34]:
# Generic Validation function
def val(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    y_pred, y_true= [], []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)


            running_loss += loss.item()
            predicted = outputs.argmax(dim = 1)

            y_true.extend(labels.cpu().tolist())
            y_pred.extend(predicted.cpu().tolist())

            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_loss = running_loss / len(list(dataloader.dataset))
    val_accuracy = accuracy = accuracy_score(y_true, y_pred)
    precision, recall, val_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro',zero_division=0)

    print(f"Val Loss: {val_loss:.6f} | Val Accuracy: {(val_accuracy * 100):.2f}% | Val F1-Score: {val_f1:.6f}")
    return val_loss, val_accuracy, val_f1


In [35]:
#Generic function for training and testing
def train_model(model, train_dataloader, val_dataloader, criterion, optimizer, scheduler, device, num_epochs):
    model=model.to(device)
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    for epoch in range(num_epochs):
        print('----------------------------------------------------------------------------')
        train_loss, train_accuracy, train_f1 = train(model, train_dataloader, criterion, optimizer, scheduler, device, epoch)
        val_loss, val_accuracy,test_f1 = val(model, val_dataloader, criterion, device)
        df.loc[epoch+1]=[epoch+1, train_accuracy, train_loss, train_f1, val_accuracy, val_loss, val_f1]
        df.to_csv('metrics.csv', index = False)
        print('\n')



    return train_loss, train_accuracy, train_f1, val_loss, val_accuracy, val_f1


## **3.3. Experimental Protocol**

Applying experimental protocol to scenarios resulting from permutations of Data Augmentations (AutoAugment and RandAugment) and Fine-Tuning:
1. Without Fine-Tuning
2. With Fine-Tuning
3. Without Fine-Tuning and with AutoAugment
4. With Fine-Tuning and with AutoAugment
5. Without Fine-Tuning and with RandAugment
6. With Fine-Tuning and with RandAugment
7. Without Fine-Tuning and with both AutoAugment and RandAugment
8. With Fine-Tuning and with both AutoAugment and RandAugment

In [44]:
data_fraction = 0.1
num_epochs=5
batch_size=32
learning_rate=0.001
num_classes = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_train_data = int((100000) * data_fraction)
num_val_data = int((20000) * data_fraction)

train_indices = random.sample(range(65000), num_train_data)
val_indices = random.sample(range(20000), num_val_data)

print(f'Number of training objects: {num_train_data}')
print(f'Number of test objects: {num_val_data}')
print("============================================================================")

train_dataloader, test_dataloader= full_data_transform(data_fraction, batch_size)
model = ConvNeXt_model()
model.head = nn.Linear(model.head.in_features, num_classes)  # Replace num_classes with the correct number of classes
num=2

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

train_loss, train_accuracy, train_f1, val_loss, val_accuracy, val_f1 = train_model(model, train_dataloader, test_dataloader,
                                                                            criterion, optimizer, scheduler, device, num_epochs)

Number of training objects: 10000
Number of test objects: 2000
----------------------------------------------------------------------------


[Epoch 1]:  12%|██████▍                                                 | 36/313 [10:34<1:21:25, 17.64s/it, loss=0.666]


KeyboardInterrupt: 

In [ ]:
print(df)

In [ ]:
df.to_csv('metrics.csv', index = False)